# N-gram 라벨링
ngram 뽑은 문장 옆에 ngram 단어쌍이 들어가는 새 열을 넣는다.

In [ ]:
import pandas as pd

# 예시 데이터프레임 생성
data = {
    '문장': [
        '자연어 처리를 배우는 것은 흥미롭다.',
        '인공지능 기술이 발전하고 있다.',
        '딥러닝은 머신러닝의 한 분야이다.'
    ]
}

df = pd.DataFrame(data)

# n-그램 생성 함수
def generate_ngrams(sentence, n):
    words = sentence.split()  # 문장을 단어로 분할
    return list(ngrams(words, n))  # n-그램 생성

# n-그램 생성 및 새로운 열에 추가
n = 2  # 예를 들어, 2-그램
df['n_grams'] = df['문장'].apply(lambda x: generate_ngrams(x, n))

# 결과 출력
print(df)